In [1]:
import pandas as pd
import numpy as np
import sklearn
import panel as pn
import re
from bokeh.plotting import figure, show
from bokeh.models import BasicTicker, ColorBar, LinearColorMapper, PrintfTickFormatter
pn.extension(comms='vscode')

ModuleNotFoundError: No module named 'panel'

In [ ]:
assets_df = pd.read_csv('data\jordans_asset_dataset_2023_03_06.csv')
assets_full_df = assets_df.copy()
assets_df.head()

,asset.cost_center,asset.u_asset_id,assigned_to,assigned_to.active,assigned_to.cost_center,assigned_to.department,assigned_to.employee_number,assigned_to.manager,BioAccess.bioFire Email Address,assigned_to.u_company_code,...,Sophos.OS,Sophos.Protection,Sophos.Tamper protection,support_group,u_ci,u_status,u_sub_status,warranty_expiration,sys_created_on,sys_created_on.1
0,2860747,AST0108027,MECHAM Jonathan,True,2860747,Software Development,16012062,MCCARTY Jameson,jonathan.mecham@biofiredx.com,028 - BioFire Diagnostics LLC,...,Windows 10 Enterprise,Intercept X Advanced with XDR,On,BMX-Deskside_SLC_L2,True,Deployed,Functional,2024-12-03 00:00:00,2022-12-09 00:00:00,2022-12-09 00:00:00
1,2858030,AST0109801,ALLEN Jared,True,2858030,BFDX IS Internal Business Software,10019166,DAVISON Kirby,Jared.Allen@biofiredx.com,028 - BioFire Diagnostics LLC,...,Windows 10 Enterprise,Intercept X Advanced with XDR,On,BMX-Deskside_SLC_L2,True,Deployed,Functional,2023-11-06 00:00:00,2022-12-09 00:00:00,2022-12-09 00:00:00
2,2860705,AST0115321,BHATIA Ajay,True,2860705,Regulatory Affairs,10018360,BOURZAC Kevin,Ajay.Bhatia@biofiredx.com,028 - BioFire Diagnostics LLC,...,Windows 10 Enterprise,Intercept X Advanced with XDR,On,BMX-Deskside_SLC_L2,True,Deployed,Functional,2025-08-16 00:00:00,2022-12-21 00:00:00,2022-12-21 00:00:00
3,2121010,AST0115322,CLARKE Kimon,True,2121010,US CLNMKT Hospital-Based Clinician,10022276,SHAFFER Matt,Kimon.Clarke@biofiredx.com,021 - bioMérieux Inc.,...,Windows 10 Enterprise,Intercept X Advanced with XDR,Off,BMX-Deskside_SLC_L2,True,Deployed,Functional,2025-08-17 00:00:00,2022-12-22 00:00:00,2022-12-22 00:00:00
4,2804740,AST0115323,SMITH Jennifer,True,2804740,Manufacturing QA Measurement Systems,10021946,BALMANNO Sam,Jennifer.Smith@biofiredx.com,028 - BioFire Diagnostics LLC,...,Windows 10 Enterprise,Intercept X Advanced with XDR,On,BMX-Deskside_SLC_L2,True,Deployed,Functional,2025-08-17 00:00:00,2022-12-22 00:00:00,2022-12-22 00:00:00


In [ ]:
for col in assets_df.columns:
    print (col)

asset.cost_center
asset.u_asset_id
assigned_to
assigned_to.active
assigned_to.cost_center
assigned_to.department
assigned_to.employee_number
assigned_to.manager
BioAccess.bioFire Email Address
assigned_to.u_company_code
BioAccess.Display Name
BioAccess.Email
Count of BioAccess.Employee ID
BioAccess.Employee Status
Sum of BioAccess.Username
category
Computer Name
BioAccess.Sunset Date
Intune.Azure AD Device ID
Intune.Azure AD registered
Intune.Category
Intune.CellularTechnology
Intune.Compliance
Intune.Computer Name
Intune.Device ID
Intune.Device state
Intune.EAS activated
Intune.EAS activation ID
Intune.EAS reason
Intune.EAS status
Intune.EID
Intune.Encrypted
Intune.Enrollment date
Intune.EthernetMAC
Intune.Free storage
Intune.ICCID
Intune.IMEI
Intune.Intune registered
Intune.Jailbroken
Intune.JoinType
Intune.Last check-in
Intune.Managed by
Intune.Management name
Intune.Manufacturer
Intune.MEID
Intune.Model
Intune.OS
Intune.OS version
Intune.Ownership
Intune.Phone number
Intune.Primary

In [ ]:
computers_by_employee_id_df = assets_df.loc[:,['assigned_to.employee_number', 'Computer Name']].groupby(['assigned_to.employee_number']).count()
computers_by_employee_id_df.columns = ["user_asset_count"]
# computers_by_employee_id_df.loc[computers_by_employee_id_df.loc[:,"user_asset_count"] == 0 ]
assets_df = assets_df.merge(computers_by_employee_id_df, on="assigned_to.employee_number")

In [ ]:
assets_df.loc[:,'Sum of BioAccess.Username']

0       16012062.0
1       10019166.0
2       10018360.0
3       10018360.0
4       10022276.0
           ...    
3050    10025949.0
3051    10019979.0
3052    10028445.0
3053    16000706.0
3054    16012438.0
Name: Sum of BioAccess.Username, Length: 3055, dtype: float64

In [ ]:
assets_df.groupby(['assigned_to.department']).count().describe()

,asset.cost_center,asset.u_asset_id,assigned_to,assigned_to.active,assigned_to.cost_center,assigned_to.employee_number,assigned_to.manager,BioAccess.bioFire Email Address,assigned_to.u_company_code,BioAccess.Display Name,...,Sophos.OS,Sophos.Protection,Sophos.Tamper protection,support_group,u_ci,u_status,u_sub_status,warranty_expiration,sys_created_on,sys_created_on.1
count,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,...,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000
mean,21.363636,21.363636,21.363636,21.363636,21.363636,21.363636,21.363636,21.363636,21.363636,21.363636,...,21.363636,21.363636,21.363636,21.363636,21.363636,21.363636,21.363636,21.363636,21.363636,21.363636
std,32.721089,32.721089,32.721089,32.721089,32.721089,32.721089,32.721089,32.721089,32.721089,32.721089,...,32.721089,32.721089,32.721089,32.721089,32.721089,32.721089,32.721089,32.721089,32.721089,32.721089
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,...,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000
75%,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,...,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
max,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,...,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000
